In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import * #optimizers
from keras.callbacks import * #earlycallback
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from commons import mean_absolute_percentage_error
from keras.layers import * #Sequentials
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from keras.utils import * #to_categorical plot_model
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
from commons import mean_absolute_percentage_error
from IPython.display import Audio
sound_file = 'beep.wav'
from keras.models import * #load_model

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [4]:
# load dataset
modelstring = 'stacked_ann/stacked_ANN_cls_pca_'#for model
dataframe = pd.read_csv("pca_75_clas.csv", sep=',')
dataframe.shape

(735, 51)

In [5]:
dataframe.head(3)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,priceUSD
0,0.074162,0.015329,-0.048046,0.042709,0.007321,-0.014251,0.001355,-0.044263,-0.014403,-0.036199,...,0.017701,-0.020600,-0.021125,-0.001148,-0.004502,-0.012360,-0.032049,0.007081,0.006557,1
1,0.094841,0.072671,-0.077840,-0.014523,0.027039,-0.053013,0.056817,-0.009060,0.047423,-0.009912,...,-0.047544,0.013065,0.065670,0.006482,0.020321,0.007130,0.016320,0.013705,-0.042491,1
2,0.064880,0.028643,-0.038454,0.019065,0.028725,-0.014173,-0.002313,-0.031474,-0.009467,-0.034115,...,0.020285,0.006481,-0.012896,0.008115,-0.022120,-0.021993,0.012241,0.021045,-0.033730,1


In [6]:
length=dataframe.shape[1]-1

In [7]:
length

50

In [8]:
# split into input (X) and output (Y) variables
X = dataframe.iloc[:,0:length]
y = dataframe['priceUSD']

In [9]:
X.head(3)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.074162,0.015329,-0.048046,0.042709,0.007321,-0.014251,0.001355,-0.044263,-0.014403,-0.036199,...,-0.004087,0.017701,-0.020600,-0.021125,-0.001148,-0.004502,-0.012360,-0.032049,0.007081,0.006557
1,0.094841,0.072671,-0.077840,-0.014523,0.027039,-0.053013,0.056817,-0.009060,0.047423,-0.009912,...,0.003421,-0.047544,0.013065,0.065670,0.006482,0.020321,0.007130,0.016320,0.013705,-0.042491
2,0.064880,0.028643,-0.038454,0.019065,0.028725,-0.014173,-0.002313,-0.031474,-0.009467,-0.034115,...,0.014521,0.020285,0.006481,-0.012896,0.008115,-0.022120,-0.021993,0.012241,0.021045,-0.033730


In [10]:
y=np.ravel(y)

In [11]:
shape=X.shape[1]

In [12]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, train_size=0.8, shuffle=False, random_state=7)

In [13]:
estimators=[]

In [14]:
estimators.append(['robust',RobustScaler()])

In [15]:
estimators.append(['mixmax',MinMaxScaler()])

In [16]:
scale=Pipeline(estimators,verbose=True)

In [17]:
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


Pipeline(memory=None,
         steps=[('robust',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ['mixmax', MinMaxScaler(copy=True, feature_range=(0, 1))]],
         verbose=True)

In [18]:
X_train=scale.transform(X_train)

In [19]:
X_test=scale.transform(X_test)

In [20]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [21]:
# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer.name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(60, activation='relu')(merge)
    hidden = Dense(10, activation='relu')(merge)
    output = Dense(2, activation='softmax')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    # plot graph of ensemble
    #plot_model(model, show_shapes=True, to_file='model_graph.png')
    # compile
    adam=optimizers.adam(lr=lr_schedule(0),amsgrad=True)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [22]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	model.fit(X, inputy_enc, epochs=300, verbose=1)

In [23]:
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [24]:
# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = modelstring + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

In [25]:
# define neural network model
def sequential_model(initializer='normal', activation='relu', neurons=350, NUM_FEATURES=shape):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    adam=optimizers.adam(lr=lr_schedule(0),amsgrad=True)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [26]:
classify=KerasClassifier(build_fn=sequential_model, batch_size=32, epochs=1000,verbose=1, shuffle=True)

In [27]:
results=cross_validate(classify,X_train,y_train,
                       cv=5,
                       scoring=('accuracy'), 
                       return_train_score=True, 
                       return_estimator=True)
Audio(sound_file,autoplay=True)

Instructions for updating:
Colocations handled automatically by placer.
Learning rate:  0.001
Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
470/470 [==============================] - 0s 348us/step - loss: 0.7318 - accuracy: 0.5128
Epoch 2/1000
470/470 [==============================] - 0s 152us/step - loss: 0.6961 - accuracy: 0.4830
Epoch 3/1000
470/470 [==============================] - 0s 138us/step - loss: 0.6895 - accuracy: 0.5234
Epoch 4/1000
470/470 [==============================] - 0s 147us/step - loss: 0.6900 - accuracy: 0.5319
Epoch 5/1000
470/470 [==============================] - 0s 155us/step - loss: 0.6867 - accuracy: 0.5255
Epoch 6/1000
470/470 [==============================] - 0s 153us/step - loss: 0.6857 - accuracy: 0.5574
Epoch 7/1000
470/470 [==============================] - 0s 169us/step - loss: 0.6849 - accuracy: 0.5511
Epoch 8/1000
470/470 [==============================] - 0s 143us/step - loss: 0.6875 - accuracy: 0.5319
Epoch 9/1000
470/470 [====

470/470 [==============================] - 0s 155us/step - loss: 0.5829 - accuracy: 0.6681
Epoch 74/1000
470/470 [==============================] - 0s 155us/step - loss: 0.5801 - accuracy: 0.6574
Epoch 75/1000
470/470 [==============================] - 0s 120us/step - loss: 0.5780 - accuracy: 0.6809
Epoch 76/1000
470/470 [==============================] - 0s 132us/step - loss: 0.5793 - accuracy: 0.6787
Epoch 77/1000
470/470 [==============================] - 0s 139us/step - loss: 0.5746 - accuracy: 0.6830
Epoch 78/1000
470/470 [==============================] - 0s 157us/step - loss: 0.5635 - accuracy: 0.6957
Epoch 79/1000
470/470 [==============================] - 0s 143us/step - loss: 0.5668 - accuracy: 0.7021
Epoch 80/1000
470/470 [==============================] - 0s 135us/step - loss: 0.5756 - accuracy: 0.6723
Epoch 81/1000
470/470 [==============================] - 0s 143us/step - loss: 0.5846 - accuracy: 0.6617
Epoch 82/1000
470/470 [==============================] - 0s 141us/ste

470/470 [==============================] - 0s 155us/step - loss: 0.4812 - accuracy: 0.7681
Epoch 151/1000
470/470 [==============================] - 0s 136us/step - loss: 0.4880 - accuracy: 0.7447
Epoch 152/1000
470/470 [==============================] - 0s 116us/step - loss: 0.4828 - accuracy: 0.7404
Epoch 153/1000
470/470 [==============================] - 0s 120us/step - loss: 0.5045 - accuracy: 0.7319
Epoch 154/1000
470/470 [==============================] - 0s 110us/step - loss: 0.4932 - accuracy: 0.7383
Epoch 155/1000
470/470 [==============================] - 0s 120us/step - loss: 0.4893 - accuracy: 0.7489
Epoch 156/1000
470/470 [==============================] - 0s 113us/step - loss: 0.4788 - accuracy: 0.7532
Epoch 157/1000
470/470 [==============================] - 0s 130us/step - loss: 0.4790 - accuracy: 0.7574
Epoch 158/1000
470/470 [==============================] - 0s 148us/step - loss: 0.4820 - accuracy: 0.7596
Epoch 159/1000
470/470 [==============================] - 0s 

470/470 [==============================] - 0s 151us/step - loss: 0.4565 - accuracy: 0.7574
Epoch 228/1000
470/470 [==============================] - 0s 153us/step - loss: 0.4209 - accuracy: 0.7745
Epoch 229/1000
470/470 [==============================] - 0s 148us/step - loss: 0.4280 - accuracy: 0.7660
Epoch 230/1000
470/470 [==============================] - 0s 144us/step - loss: 0.4435 - accuracy: 0.7745
Epoch 231/1000
470/470 [==============================] - 0s 160us/step - loss: 0.4214 - accuracy: 0.7851
Epoch 232/1000
470/470 [==============================] - 0s 113us/step - loss: 0.4233 - accuracy: 0.7745
Epoch 233/1000
470/470 [==============================] - 0s 124us/step - loss: 0.4223 - accuracy: 0.7787
Epoch 234/1000
470/470 [==============================] - 0s 112us/step - loss: 0.4236 - accuracy: 0.7787
Epoch 235/1000
470/470 [==============================] - 0s 109us/step - loss: 0.4246 - accuracy: 0.7745
Epoch 236/1000
470/470 [==============================] - 0s 

470/470 [==============================] - 0s 162us/step - loss: 0.3948 - accuracy: 0.8043
Epoch 305/1000
470/470 [==============================] - 0s 142us/step - loss: 0.4068 - accuracy: 0.7872
Epoch 306/1000
470/470 [==============================] - 0s 156us/step - loss: 0.3962 - accuracy: 0.7915
Epoch 307/1000
470/470 [==============================] - 0s 143us/step - loss: 0.4087 - accuracy: 0.7851
Epoch 308/1000
470/470 [==============================] - 0s 144us/step - loss: 0.4094 - accuracy: 0.7681
Epoch 309/1000
470/470 [==============================] - 0s 140us/step - loss: 0.4381 - accuracy: 0.7468
Epoch 310/1000
470/470 [==============================] - 0s 139us/step - loss: 0.4114 - accuracy: 0.7809
Epoch 311/1000
470/470 [==============================] - 0s 129us/step - loss: 0.3918 - accuracy: 0.7915
Epoch 312/1000
470/470 [==============================] - 0s 150us/step - loss: 0.4448 - accuracy: 0.7787
Epoch 313/1000
470/470 [==============================] - 0s 

...

471/471 [==============================] - 0s 115us/step - loss: 0.2006 - accuracy: 0.9002
Epoch 690/1000
471/471 [==============================] - 0s 120us/step - loss: 0.2010 - accuracy: 0.9066
Epoch 691/1000
471/471 [==============================] - 0s 110us/step - loss: 0.1944 - accuracy: 0.9045
Epoch 692/1000
471/471 [==============================] - 0s 110us/step - loss: 0.2039 - accuracy: 0.9108
Epoch 693/1000
471/471 [==============================] - 0s 114us/step - loss: 0.2038 - accuracy: 0.8938
Epoch 694/1000
471/471 [==============================] - 0s 111us/step - loss: 0.1965 - accuracy: 0.9023
Epoch 695/1000
471/471 [==============================] - 0s 112us/step - loss: 0.1908 - accuracy: 0.9087
Epoch 696/1000
471/471 [==============================] - 0s 117us/step - loss: 0.2112 - accuracy: 0.8875
Epoch 697/1000
471/471 [==============================] - 0s 112us/step - loss: 0.2168 - accuracy: 0.8896
Epoch 698/1000
471/471 [==============================] - 0s 

471/471 [==============================] - 0s 115us/step - loss: 0.1854 - accuracy: 0.9151
Epoch 767/1000
471/471 [==============================] - 0s 116us/step - loss: 0.1880 - accuracy: 0.9151
Epoch 768/1000
471/471 [==============================] - 0s 119us/step - loss: 0.1882 - accuracy: 0.9108
Epoch 769/1000
471/471 [==============================] - 0s 111us/step - loss: 0.1921 - accuracy: 0.9151
Epoch 770/1000
471/471 [==============================] - 0s 121us/step - loss: 0.1851 - accuracy: 0.9087
Epoch 771/1000
471/471 [==============================] - 0s 110us/step - loss: 0.1911 - accuracy: 0.9023
Epoch 772/1000
471/471 [==============================] - 0s 113us/step - loss: 0.1881 - accuracy: 0.9130
Epoch 773/1000
471/471 [==============================] - 0s 117us/step - loss: 0.1941 - accuracy: 0.9066
Epoch 774/1000
471/471 [==============================] - 0s 115us/step - loss: 0.2131 - accuracy: 0.8981
Epoch 775/1000
471/471 [==============================] - 0s 

471/471 [==============================] - 0s 110us/step - loss: 0.1908 - accuracy: 0.9066
Epoch 844/1000
471/471 [==============================] - 0s 109us/step - loss: 0.1807 - accuracy: 0.9257
Epoch 845/1000
471/471 [==============================] - 0s 114us/step - loss: 0.1797 - accuracy: 0.9108
Epoch 846/1000
471/471 [==============================] - 0s 112us/step - loss: 0.1718 - accuracy: 0.9172
Epoch 847/1000
471/471 [==============================] - 0s 110us/step - loss: 0.1875 - accuracy: 0.9023
Epoch 848/1000
471/471 [==============================] - 0s 107us/step - loss: 0.2146 - accuracy: 0.9023
Epoch 849/1000
471/471 [==============================] - 0s 109us/step - loss: 0.2049 - accuracy: 0.8981
Epoch 850/1000
471/471 [==============================] - 0s 111us/step - loss: 0.1856 - accuracy: 0.9108
Epoch 851/1000
471/471 [==============================] - 0s 111us/step - loss: 0.1873 - accuracy: 0.9108
Epoch 852/1000
471/471 [==============================] - 0s 

471/471 [==============================] - 0s 117us/step - loss: 0.1740 - accuracy: 0.9236
Epoch 921/1000
471/471 [==============================] - 0s 112us/step - loss: 0.1702 - accuracy: 0.9193
Epoch 922/1000
471/471 [==============================] - 0s 109us/step - loss: 0.1827 - accuracy: 0.9151
Epoch 923/1000
471/471 [==============================] - 0s 113us/step - loss: 0.1827 - accuracy: 0.9087
Epoch 924/1000
471/471 [==============================] - 0s 117us/step - loss: 0.1700 - accuracy: 0.9214
Epoch 925/1000
471/471 [==============================] - 0s 108us/step - loss: 0.1708 - accuracy: 0.9172
Epoch 926/1000
471/471 [==============================] - 0s 119us/step - loss: 0.1715 - accuracy: 0.9236
Epoch 927/1000
471/471 [==============================] - 0s 109us/step - loss: 0.1689 - accuracy: 0.9151
Epoch 928/1000
471/471 [==============================] - 0s 108us/step - loss: 0.1729 - accuracy: 0.9172
Epoch 929/1000
471/471 [==============================] - 0s 

471/471 [==============================] - 0s 111us/step - loss: 0.1638 - accuracy: 0.9342
Epoch 998/1000
471/471 [==============================] - 0s 113us/step - loss: 0.1594 - accuracy: 0.9257
Epoch 999/1000
471/471 [==============================] - 0s 129us/step - loss: 0.1621 - accuracy: 0.9214
Epoch 1000/1000
471/471 [==============================] - 0s 35us/step


In [28]:
results['train_score']

array([0.84893617, 0.86808511, 0.92553191, 0.92356688, 0.92993631])

In [29]:
results['test_score']

array([0.50847458, 0.57627119, 0.49152542, 0.43589744, 0.54700855])

In [30]:
results['estimator']

(<keras.wrappers.scikit_learn.KerasClassifier at 0x7f06e620ed68>,
 <keras.wrappers.scikit_learn.KerasClassifier at 0x7f06ae5575f8>)

In [31]:
# create directory for models
dir = 'stacked_ann'
if os.path.exists(dir):
    shutil.rmtree(dir)
os.makedirs(dir)

In [32]:
# save models
n_members = len(results['estimator'])
for i in range(n_members):
    # save model
    filename = modelstring + str(i + 1) + '.h5'
    results['estimator'][i].model.save(filename)
    print('>Saved %s' % filename)

>Saved stacked_ann/stacked_ANN_cls_pca_1.h5
>Saved stacked_ann/stacked_ANN_cls_pca_2.h5
>Saved stacked_ann/stacked_ANN_cls_pca_3.h5
>Saved stacked_ann/stacked_ANN_cls_pca_4.h5
>Saved stacked_ann/stacked_ANN_cls_pca_5.h5


In [33]:
# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded stacked_ann/stacked_ANN_cls_pca_1.h5
>loaded stacked_ann/stacked_ANN_cls_pca_2.h5
>loaded stacked_ann/stacked_ANN_cls_pca_3.h5
>loaded stacked_ann/stacked_ANN_cls_pca_4.h5
>loaded stacked_ann/stacked_ANN_cls_pca_5.h5
Loaded 5 models


In [34]:
# define ensemble model
stacked_model = define_stacked_model(members)
# fit stacked model on test dataset
fit_stacked_model(stacked_model,X_test, y_test)
Audio(sound_file,autoplay=True)

Learning rate:  0.001
Epoch 1/300
147/147 [==============================] - 0s 1ms/step - loss: 0.7177 - accuracy: 0.5170
Epoch 2/300
147/147 [==============================] - 0s 169us/step - loss: 0.7124 - accuracy: 0.5102
Epoch 3/300
147/147 [==============================] - 0s 144us/step - loss: 0.7068 - accuracy: 0.5170
Epoch 4/300
147/147 [==============================] - 0s 243us/step - loss: 0.7035 - accuracy: 0.5238
Epoch 5/300
147/147 [==============================] - 0s 164us/step - loss: 0.7010 - accuracy: 0.5170
Epoch 6/300
147/147 [==============================] - 0s 173us/step - loss: 0.6983 - accuracy: 0.5170
Epoch 7/300
147/147 [==============================] - 0s 220us/step - loss: 0.6966 - accuracy: 0.5238
Epoch 8/300
147/147 [==============================] - 0s 165us/step - loss: 0.6949 - accuracy: 0.5238
Epoch 9/300
147/147 [==============================] - 0s 174us/step - loss: 0.6937 - accuracy: 0.5306
Epoch 10/300
147/147 [==============================]

147/147 [==============================] - 0s 133us/step - loss: 0.6595 - accuracy: 0.5918
Epoch 80/300
147/147 [==============================] - 0s 220us/step - loss: 0.6587 - accuracy: 0.5850
Epoch 81/300
147/147 [==============================] - 0s 151us/step - loss: 0.6586 - accuracy: 0.5918
Epoch 82/300
147/147 [==============================] - 0s 136us/step - loss: 0.6583 - accuracy: 0.5850
Epoch 83/300
147/147 [==============================] - 0s 154us/step - loss: 0.6582 - accuracy: 0.5850
Epoch 84/300
147/147 [==============================] - 0s 128us/step - loss: 0.6578 - accuracy: 0.5850
Epoch 85/300
147/147 [==============================] - 0s 157us/step - loss: 0.6577 - accuracy: 0.5850
Epoch 86/300
147/147 [==============================] - 0s 134us/step - loss: 0.6573 - accuracy: 0.5782
Epoch 87/300
147/147 [==============================] - 0s 126us/step - loss: 0.6570 - accuracy: 0.5850
Epoch 88/300
147/147 [==============================] - 0s 148us/step - loss:

147/147 [==============================] - 0s 142us/step - loss: 0.6459 - accuracy: 0.5714
Epoch 158/300
147/147 [==============================] - 0s 131us/step - loss: 0.6458 - accuracy: 0.5714
Epoch 159/300
147/147 [==============================] - 0s 157us/step - loss: 0.6456 - accuracy: 0.5714
Epoch 160/300
147/147 [==============================] - 0s 126us/step - loss: 0.6455 - accuracy: 0.5714
Epoch 161/300
147/147 [==============================] - 0s 136us/step - loss: 0.6455 - accuracy: 0.5714
Epoch 162/300
147/147 [==============================] - 0s 154us/step - loss: 0.6453 - accuracy: 0.5714
Epoch 163/300
147/147 [==============================] - 0s 123us/step - loss: 0.6453 - accuracy: 0.5714
Epoch 164/300
147/147 [==============================] - 0s 138us/step - loss: 0.6455 - accuracy: 0.5714
Epoch 165/300
147/147 [==============================] - 0s 127us/step - loss: 0.6450 - accuracy: 0.5782
Epoch 166/300
147/147 [==============================] - 0s 149us/ste

147/147 [==============================] - 0s 153us/step - loss: 0.6391 - accuracy: 0.5850
Epoch 236/300
147/147 [==============================] - 0s 125us/step - loss: 0.6390 - accuracy: 0.5850
Epoch 237/300
147/147 [==============================] - 0s 133us/step - loss: 0.6390 - accuracy: 0.5986
Epoch 238/300
147/147 [==============================] - 0s 127us/step - loss: 0.6389 - accuracy: 0.5918
Epoch 239/300
147/147 [==============================] - 0s 126us/step - loss: 0.6390 - accuracy: 0.5986
Epoch 240/300
147/147 [==============================] - 0s 149us/step - loss: 0.6388 - accuracy: 0.5986
Epoch 241/300
147/147 [==============================] - 0s 123us/step - loss: 0.6388 - accuracy: 0.5986
Epoch 242/300
147/147 [==============================] - 0s 197us/step - loss: 0.6387 - accuracy: 0.5986
Epoch 243/300
147/147 [==============================] - 0s 139us/step - loss: 0.6386 - accuracy: 0.5918
Epoch 244/300
147/147 [==============================] - 0s 145us/ste

In [35]:
stacked_model.save('trained_models/stacked_ANN_cls_pca.hdf5')

In [36]:
stacked_model=load_model('trained_models/stacked_ANN_cls_pca.hdf5',compile=False)

In [37]:
# make predictions
y_pred = predict_stacked_model(stacked_model, X_test)

In [38]:
y_pred=np.argmax(y_pred,axis=1)

In [39]:
accuracy_score(np.ravel(y_test),y_pred)

0.6122448979591837

In [43]:
f1_score(y_test,y_pred,average='weighted')

0.60798205117142

In [44]:
auc=roc_auc_score(y_test,y_pred)
auc

0.6087842846553002

In [41]:
print(classification_report(y_test,y_pred,labels=[0,1], target_names=['decrease','increase']))

              precision    recall  f1-score   support

    decrease       0.88      0.29      0.43       259
    increase       0.53      0.95      0.68       222

    accuracy                           0.59       481
   macro avg       0.71      0.62      0.56       481
weighted avg       0.72      0.59      0.55       481



In [42]:
pd.DataFrame(zip(y_test,y_pred),columns=['y_test','y_pred'])

,y_test,y_pred
0,0,0
1,0,0
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,0,1
8,0,1
9,0,1
